In [ ]:
#!/usr/bin/env python
# coding: utf-8
import re,time,os
from selenium import webdriver
from pony.orm import *
db = Database()
#定义数据模型
class Ttitle(db.Entity):
    lx = Optional(str)
    bh = PrimaryKey(str, auto=True)
    url = Optional(str)
    bt = Optional(str)
    fs = Optional(str)
    tgs = Optional(str)
    tjs = Optional(str)
    tgl = Optional(str)
    sj = Optional(str)
def create_database(dbpath):
    """创建数据库，如果数据库文件存在则删除之后再创建"""
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath,"w")
    f.close()
    #db.generate_mapping()
    db.bind(provider='sqlite', filename=dbpath)#provider是数据库的类型,filename是文件的路径
    db.generate_mapping(create_tables=True)
    #将实体映射到数据库表,参数create_tables = True表示如果实体指向的表尚不存在，则使用CREATE TABLE命令创建它们。必须在调用generate_mapping（）方法之前定义连接到数据库的所有实体。
    set_sql_debug(True)
def get_html(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    driver.close()
    return html

def extract_list_info(url,category):
    """抽取列表页的信息"""
    html = get_html(url)
    result = []
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href="([\d\D]*?)" class="">([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td></tr>')
    for match in reobj.finditer(html):#获取列表页的标号、链接、标题、分数、通过数、提交数、通过率
        biaohao = match.group(1)
        lianjie = "https://pintia.cn/" + match.group(2)
        biaoti = match.group(3)
        fenshu = match.group(4)
        tongguoshu = match.group(5)
        tijiaosshu = match.group(6)
        tongguolv = match.group(7)
        for a,b,c,d,e,f,g in zip(biaohao,lianjie,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv):#打包在一起
            result.append((category,a,b,c,d,e,f,g))
    return result#返回元组列表

def extract_detail_info(results):
    """抽取详情页数据"""
    list1 = []
    final_result = []
    for i in results:
        url1=results[2]#从中获取出链接
        html1 = get_html(url1)
        reobj = re.compile(r'<div class="problem_Yd8rq p-3 mb-3 background-grey-1"><div class="ques-view">([\d\D]*?)</div><div class="problemInfo_OZ-Rv">')
        for match in reobj.finditer(html1):
            list1.append(match.group(1))
    for x,y in zip(results,list1):#将详情页的内容与列表页内容拼接
        final_result.append((x,y))
    return final_result#返回元组列表

@db_session
def store(final_results):
    """进行数据存储""""
    for x in final_results:#循环写入到program.sqlite文件中去
        s = Ttitle(lx=x[0],bh=x[1],url=x[2],bt=x[3],fs=x[4],tgs=x[5],tjs=x[6],tgl=x[7],sj=x[8])
        db.commit()
        #Pony不会立即在数据库中保存对象。只有在调用commit()函数后才会保存这些对象。如果打开调试模式，那么在commit（）期间，您将看到发送到数据库的五个insert命令。


def main():
    """主函数"""
    create_database('D:/code2020/program.sqlite')
    final_results = []
    urls = [("https://pintia.cn/problem-sets/1111652100718116864/problems/type/6","函数题"),("https://pintia.cn/problem-sets/1111652100718116864/problems/type/7","编程题")]
    for url,category in urls:   
        results = extract_list_info(url,category)#获取列表页的内容
        results = extract_detail_info(results)#获取详情页面的内容,并且与列表信息合并
        final_results.extend(results)
    store(final_results)


# if __name__ == "__main___":
#     main()

main()